In [1]:
using CSV
using DataFrames
using Dates
using Gadfly

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
rawdata = DataFrame(CSV.File("../rawdata/reported_hospital_capacity_admissions_facility_level_weekly_average_timeseries_20210214.csv"));

In [4]:
bad_ids = begin
    id_counts = combine(groupby(rawdata, :hospital_pk), :hospital_name => (x -> length(unique(x))) => :n_names)
    filter!(x -> x.n_names > 1, id_counts)
    unique(id_counts.hospital_pk)
end
filter!(row -> !(row.hospital_pk in bad_ids), rawdata);

In [5]:
data = select(rawdata,
    :hospital_name => :hospital,
    :hospital_pk => :hospital_id,
    :collection_week => :date,
    :all_adult_hospital_inpatient_beds_7_day_avg => ByRow(x -> (ismissing(x) || x == -999999) ? missing : x) => :beds_allbeds,
    :total_staffed_adult_icu_beds_7_day_avg => ByRow(x -> (ismissing(x) || x == -999999) ? missing : x) => :beds_icu,
);
data.beds_acute = max.(0, data.beds_allbeds - data.beds_icu)
sort!(data, [:hospital, :hospital_id, :date]);

In [6]:
start_date = Date(2020,  7, 1)
end_date   = Date(2020, 11, 1)
date_range = collect(start_date : Day(1) : end_date);

In [7]:
function mean_capacity(xs, dates)
    xs = [x for (x,d) in zip(xs,dates) if d in date_range]
    if all(ismissing.(xs))
        return 0
    else
        xs = filter(x -> !ismissing(x), xs)
        return round(Int, sum(xs) / length(xs))
    end
end;

function max_capacity(xs)
    if all(ismissing.(xs))
        return 0
    else
        return maximum(skipmissing(xs))
    end
end;

function min_capacity(xs)
    if all(ismissing.(xs))
        return 0
    else
        return minimum(skipmissing(xs))
    end
end;

In [8]:
capacity_data = combine(groupby(data, [:hospital, :hospital_id]), [
    :beds_icu => min_capacity => :beds_icu_min,
    :beds_acute => min_capacity => :beds_acute_min,
    :beds_allbeds => min_capacity => :beds_allbeds_min,

    [:beds_icu, :date] => mean_capacity => :beds_icu_mean,
    [:beds_acute, :date] => mean_capacity => :beds_acute_mean,
    [:beds_allbeds, :date] => mean_capacity => :beds_allbeds_mean,
        
    :beds_icu => max_capacity => :beds_icu_max,
    :beds_acute => max_capacity => :beds_acute_max,
    :beds_allbeds => max_capacity => :beds_allbeds_max,
]);

In [9]:
capacity_names_icu = [:beds_icu_min, :beds_icu_mean, :beds_icu_max];
capacity_names_acute = [:beds_acute_min, :beds_acute_mean, :beds_acute_max];
capacity_names_allbeds = [:beds_allbeds_min, :beds_allbeds_mean, :beds_allbeds_max];

In [10]:
sort!(capacity_data, [:hospital, :hospital_id]);

In [11]:
first(capacity_data, 5)

,hospital,hospital_id,beds_icu_min,beds_acute_min,beds_allbeds_min,beds_icu_mean,beds_acute_mean,beds_allbeds_mean,beds_icu_max,beds_acute_max,beds_allbeds_max
,String,String,Float64,Float64,Float64,Int64,Int64,Int64,Float64,Float64,Float64
1,ABBEVILLE AREA MEDICAL CENTER,421301,0.0,10.6,10.6,0,16,16,0.0,18.0,18.0
2,ABBEVILLE GENERAL HOSPITAL,190034,5.6,27.0,32.6,7,31,38,7.0,31.0,38.0
3,ABBOTT NORTHWESTERN HOSPITAL,240057,39.4,352.6,406.1,51,614,665,63.3,642.4,693.1
4,ABILENE REGIONAL MEDICAL CENTER,450558,9.3,51.4,66.0,14,68,82,21.3,86.9,104.4
5,ABINGTON MEMORIAL HOSPITAL,390231,52.0,396.0,460.0,55,466,521,64.0,481.0,545.0


In [12]:
capacity_data_long = stack(capacity_data, Not([:hospital, :hospital_id]), variable_name=:capacity_name, value_name=:capacity_value);

In [13]:
capacity_data_long_icu = filter(row -> Symbol(row.capacity_name) in capacity_names_icu, capacity_data_long);
capacity_data_long_acute = filter(row -> Symbol(row.capacity_name) in capacity_names_acute, capacity_data_long);
capacity_data_long_allbeds = filter(row -> Symbol(row.capacity_name) in capacity_names_allbeds, capacity_data_long);

In [14]:
capacity_data_output = select(capacity_data, :hospital, :hospital_id, :beds_icu_mean => :capacity_icu, :beds_acute_mean => :capacity_acute, :beds_allbeds_mean => :capacity_allbeds);

In [15]:
filter!(row -> row.capacity_icu + row.capacity_acute + row.capacity_allbeds > 0, capacity_data_output);

In [16]:
capacity_data_output |> CSV.write("../data/capacity_hhs.csv");